In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import os
from multiprocessing import Pool




In [2]:
#scrape root directory from physionet MIMICIII waveform (only)
ROOTURL = "https://physionet.org/files/mimic3wdb/1.0"
RecordsRequest = requests.get(ROOTURL + "/RECORDS-waveforms")

#convert request item in Soup obj and into vector of string names
soup = BeautifulSoup(RecordsRequest.text, 'html.parser')
all_record_paths = str(soup).splitlines();

In [6]:
idx = 0;
def scrape_mimic(record_path):
    record_name = record_path[3:10]

    #Will look in each record's general layout file for presence of an ABP and PLETH waveform
    #    if it exists then will download file
    LayoutRequest = requests.get(ROOTURL + "/" + record_path + record_name+"_layout.hea")
    soup_layout = BeautifulSoup(LayoutRequest.text, 'html.parser')
    layout_str = str(soup_layout)
    signal_matches = ["ABP", "PLETH"] 

    if not all( x in layout_str for x in signal_matches):
        return -1
    
    #use --no-parent otherwise you will download all directory files
    download_cmd = "wget -r --no-parent " + ROOTURL + "/" + record_path;
    print("DOWNLOADING -- " + record_name )
    os.system(download_cmd)
    print("DOWNLOAD COMPLETE -- " + record_name)
    
#     idx = idx+1;
#     if idx>=2: return -1;


    
if __name__ == '__main__':
    with Pool(8) as p:
        p.map(scrape_mimic, all_record_paths)

DOWNLOADING -- 3092779
DOWNLOADING -- 3186294
DOWNLOADING -- 3155355
DOWNLOADING -- 3031061
DOWNLOADING -- 3123427
DOWNLOADING -- 3000063
DOWNLOADING -- 3061871
DOWNLOADING -- 3217258
DOWNLOAD COMPLETE -- 3031061
DOWNLOADING -- 3031101
DOWNLOAD COMPLETE -- 3155355
DOWNLOADING -- 3155448
DOWNLOAD COMPLETE -- 3092779
DOWNLOADING -- 3092783
DOWNLOAD COMPLETE -- 3123427
DOWNLOADING -- 3123428
DOWNLOAD COMPLETE -- 3061871
DOWNLOAD COMPLETE -- 3000063
DOWNLOADING -- 3061909
DOWNLOAD COMPLETE -- 3092783
DOWNLOADING -- 3092827
DOWNLOAD COMPLETE -- 3217258
DOWNLOADING -- 3217304
DOWNLOAD COMPLETE -- 3186294
DOWNLOADING -- 3186307
DOWNLOADING -- 3000393
DOWNLOAD COMPLETE -- 3123428
DOWNLOADING -- 3123518
DOWNLOAD COMPLETE -- 3092827
DOWNLOADING -- 3092990
DOWNLOAD COMPLETE -- 3123518
DOWNLOADING -- 3123610
DOWNLOAD COMPLETE -- 3217304
DOWNLOADING -- 3217329
DOWNLOAD COMPLETE -- 3061909
DOWNLOADING -- 3062010
DOWNLOAD COMPLETE -- 3092990
DOWNLOADING -- 3092993
DOWNLOAD COMPLETE -- 3186307
DOWNLOA

Process ForkPoolWorker-13:


DOWNLOAD COMPLETE -- 3155561


KeyboardInterrupt: 

In [6]:
RecordsRequest

<Response [200]>

In [16]:
all_records

['30/3000003/',
 '30/3000051/',
 '30/3000060/',
 '30/3000063/',
 '30/3000065/',
 '30/3000086/',
 '30/3000100/',
 '30/3000103/',
 '30/3000105/',
 '30/3000125/',
 '30/3000126/',
 '30/3000142/',
 '30/3000154/',
 '30/3000189/',
 '30/3000190/',
 '30/3000203/',
 '30/3000221/',
 '30/3000282/',
 '30/3000336/',
 '30/3000358/',
 '30/3000386/',
 '30/3000393/',
 '30/3000397/',
 '30/3000428/',
 '30/3000435/',
 '30/3000458/',
 '30/3000480/',
 '30/3000484/',
 '30/3000497/',
 '30/3000531/',
 '30/3000544/',
 '30/3000577/',
 '30/3000596/',
 '30/3000598/',
 '30/3000611/',
 '30/3000686/',
 '30/3000701/',
 '30/3000710/',
 '30/3000713/',
 '30/3000714/',
 '30/3000715/',
 '30/3000716/',
 '30/3000717/',
 '30/3000724/',
 '30/3000762/',
 '30/3000775/',
 '30/3000781/',
 '30/3000801/',
 '30/3000834/',
 '30/3000847/',
 '30/3000855/',
 '30/3000858/',
 '30/3000860/',
 '30/3000866/',
 '30/3000878/',
 '30/3000879/',
 '30/3000885/',
 '30/3000912/',
 '30/3000960/',
 '30/3000983/',
 '30/3000989/',
 '30/3001002/',
 '30/300

In [8]:
all_record_paths[0]

'30/3000003/'

In [21]:
import threading
import time

exitFlag = 0

class myThread (threading.Thread):
   def __init__(self, threadID, name, counter):
      threading.Thread.__init__(self)
      self.threadID = threadID
      self.name = name
      self.counter = counter
   def run(self):
      print( "Starting " + self.name)
      print(123)
      print( "Exiting " + self.name)

def print_time(threadName, counter, delay):
   while counter:
      if exitFlag:
         threadName.exit()
      time.sleep(delay)
      print ("%s: %s" % (threadName, time.ctime(time.time())))
      counter -= 1
    
print(124)
# Create new threads
thread1 = myThread(1, "Thread-1", 1)
thread2 = myThread(2, "Thread-2", 2)

# Start new Threads
thread1.start()
thread2.start()

print( "Exiting Main Thread")

124
Starting Thread-1Starting Thread-2
123
Exiting Thread-1
Exiting Main Thread

123
Exiting Thread-2


In [23]:
print(all_record_paths[0:len(all_record_paths):5])

['30/3000003/', '30/3000086/', '30/3000126/', '30/3000203/', '30/3000386/', '30/3000458/', '30/3000544/', '30/3000686/', '30/3000715/', '30/3000775/', '30/3000855/', '30/3000879/', '30/3000989/', '30/3001091/', '30/3001158/', '30/3001203/', '30/3001302/', '30/3001445/', '30/3001554/', '30/3001572/', '30/3001646/', '30/3001700/', '30/3001775/', '30/3001810/', '30/3001909/', '30/3001964/', '30/3002069/', '30/3002151/', '30/3002221/', '30/3002270/', '30/3002324/', '30/3002357/', '30/3002507/', '30/3002541/', '30/3002562/', '30/3002634/', '30/3002666/', '30/3002762/', '30/3002792/', '30/3002874/', '30/3002921/', '30/3003030/', '30/3003127/', '30/3003211/', '30/3003269/', '30/3003339/', '30/3003402/', '30/3003467/', '30/3003616/', '30/3003706/', '30/3003801/', '30/3003908/', '30/3003965/', '30/3004021/', '30/3004063/', '30/3004110/', '30/3004172/', '30/3004196/', '30/3004229/', '30/3004281/', '30/3004325/', '30/3004448/', '30/3004555/', '30/3004658/', '30/3004751/', '30/3004816/', '30/30048